# Projecting images from a PNG or JPEG file

First import the modules

This notebook requires the ipywidgets for manual image fitting. Please see the documentation [here](https://ipywidgets.readthedocs.io/en/stable/user_install.html) for instructions to install it. You will also need `numpy`, `scipy`, `matplotlib`, `spiceypy` and `astropy`, which can be installed using the `pip` tool. 

In [1]:
%matplotlib widget
from projector import Projector

Then, we are going to initialize the object with the filepath, date, path to NAIF SPICE kernels and the target name. The input is defined as follows:

### Date
The date must be in UTC and ISO format (yyyy-mm-dd hh:mm:ss or yyyy-mm-ddThh:mm:ss). 

### Kernel path
This is the path to the generic_kernels folder provided by NAIF (see description [here](https://naif.jpl.nasa.gov/naif/data_generic.html)). You can dowload this data to any folder by running the following command in a UNIX shell:

```bash
wget -m -nH --cut-dirs=2 -nv ftp://naif.jpl.nasa.gov/pub/naif/generic_kernels/
```
This will create the `generic_kernels` folder in the current directory, which is the location that `kernel_path` should point to.

###  Target
The name of the target. Use all-caps (e.g. 'JUPITER')

In [3]:
KERNEL_DIR = '/home/local/Isis/data/generic_kernels/'

date = '2019-02-02 10:07'
proj = Projector.from_img('2019-02-02-1007_1-RGBdp.jpg', date, KERNEL_DIR, target='JUPITER')

## Calibrating the camera plate
There are two ways to do this: 

* Automatically determining the plate scale and rotation from the image. This works well for Jupiter, but is not recommended for Saturn/Mars. Use the `fit_img_ellipse` method for this route.

* Manually aligning the observed limb to match the image. In this case, use the `fit_img_manual` method.

### Automated fitting
Here, we call the `fit_img_ellipse` method, which automatically adjusts the plate scale and rotation to match the shape of the observed elliptic limb of the object. The input parameters are:

* gamma : the contrast enhancment factor to determine the location of the limb. Adjusts the pixel value by the power of `gamma`. Default value of 0.1 works quite well for Jupiter.

* threshold : The pixel value threshold (from 0 to 1) to detect the limb. With the default value of `gamma`, the planet usually has a pixel value of greater than 1, so anything between 0.1-0.9 works well. Adjust if the image is too noisy. 

* NorthUp : A boolean flag to specify that the north is up in the image (default `True`). Toggle to `False` if the image is aligned south pole up.

On completion, you will see the image with a red line highlighting the observed limb, a while line showing the fitted limb. Ideally these should overlap with a good fit. Two blue lines will show the equator and meridian at the sub-observer longitude. A green circle shows the north pole and green cross shows the south pole. Two extra green lines are plotted at 24$^\circ$ N and 22$^\circ$ S, as they usually correspond to prominent Belt/Zone boundaries on Jupiter.

In [4]:
proj.fit_img_ellipse(NorthUp=True)

/usr/local/lib/python3.6/dist-packages/matplotlib/__init__.py:941: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

### Manual image fitting
For other planets, or if the fit does not produce the required accuracy, you can tweak the fit manually. Here we call the `fit_img_manual` function, which provides an image of the observed and fit limb, and four parameters for calibrating the plate:

* `scale` : this is the plate scale in arcsec/pixel. This fit assumes that the plate scale in both direction is constant. 

* `alpha` : the rotation of the J2000 north from the camera +y axis (which is the bottom of the image)

* `x0` : x-coordinate of the target center in pixels

* `y0` : y-coordinate of the target center in pixels

The `gamma` slider is to adjust the image contrast to see the limb better.

Adjust the sliders until the image limb matches the fitted limb. As before, the meridian is plotted in blue and north pole is the white circle and the south pole is the white cross. 

The `fit_img_manual` function takes four optional arguments, which are the limits for the slider for `scale`, `alpha`, `x0` and `y0`. These are useful for fine adjustment of the calibration parameters. Once you have a good guess for any, you can input these as tuples. For e.g.:
```python
proj.fit_img_manual(scalelim=(10., 12.)) ## limit the slider between 10-12 "/pix
```
The code will preserve the value of the slider, so you can rerun the cell iteratively as you constrain the limits. 

In [5]:
proj.fit_img_manual()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

interactive(children=(FloatSlider(value=1.0, description='gamma', max=1.0, min=0.1, step=0.01), Output()), _do…

interactive(children=(FloatSlider(value=18.711655449170152, description='scale', max=25.0, min=5.0, step=0.01)…

## Projecting to a latitude/longitude map

Once the plate is calibrated, we can determine the coordinate values of each pixel. This is done using the `project` method. This method takes two optional parameters:

* `get_illum` : boolean value to also get illumination values (incidence, emission and phase angles, and an I/F correction using the Lambertian model. The default value is `True` and these parameters can be retrieved from the `proj` object via `proj.incidence`, `proj.emission`, `proj.phase` and `proj.solar_corr` respectively. Turn off to speed up the fit. Multiply the raw image by `solar_corr` to get the corrected image. 

* `use_full_image` : boolean value to check the entire image. If turned off (default is `False`), then only the pixels $1.2\times$ the target radii are determined, in order to speed up computation. Toggle on if the bounding box is incorrect. 

The method will print a progress bar as it scans over the y-coordinate and produces a longitude and latitude map along with a grid over the image. If `get_illum` was enabled, this will plot the corrected image. 

In [6]:
proj.project()

 800/810

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


## Saving the data

**Coming soon....**

The latitude and longitude values are saved to the `proj.lat` and `proj.lon` respectively. These can now be saved in any format. 